# Part 1

In [1]:
# !pip install pyspark
%ls -la

total 2931336
drwxrwxrwx 1 root   root        4096 Dec  1 13:47 ./
drwxr-xr-x 1 root   root        4096 Nov 20 11:30 ../
-rw------- 1 jovyan users        111 Nov 30 07:44 .bash_history
drwxr-xr-x 1 jovyan users       4096 Nov 24 11:00 .cache/
drwxr-xr-x 1 jovyan users       4096 Nov 24 11:04 .ipynb_checkpoints/
drwxr-xr-x 1 jovyan users       4096 Nov 24 11:04 .ipython/
drwxr-xr-x 1 jovyan users       4096 Nov 30 07:40 .jupyter/
-rw-r--r-- 1 jovyan users      21107 Dec  1 13:47 lab8.ipynb
drwxr-xr-x 1 jovyan users       4096 Nov 24 10:59 .local/
drwxr-xr-x 1 jovyan users       4096 Nov 24 11:35 spark-warehouse/
-rwxrwxrwx 1 root   root  3001659271 Dec 19  2011 train_triplets.txt*
drwxrwxrwx 1 root   root        4096 Nov 24 10:20 yelp-dataset/


In [2]:
import pyspark


In [3]:
from pyspark.sql import SparkSession

# since everyone will be using cluster at the same time
# let's make sure that everyone has resource. that is why 
# the configuration uses dynamic resource allocation and
# maximum 1 executor 
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.dynamicAllocation.enabled", "true")\
    .config("spark.dynamicAllocation.shuffleTracking.enabled", "true")\
    .config("spark.dynamicAllocation.maxExecutors", "1")\
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/01 13:48:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
songs_df = spark.read.load("./train_triplets.txt",
                     format="csv", sep="\t", inferSchema="true", 
                     header="false")

In [5]:
songs_df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: integer (nullable = true)



In [6]:
songs_df = songs_df.withColumnRenamed("_c0", "user")\
                   .withColumnRenamed("_c1", "song")\
                   .withColumnRenamed("_c2", "play_count")

In [7]:
songs_df.createOrReplaceTempView("songs")

In [8]:
played_more_than_10_times = spark.sql("select song from songs where play_count > 10")

In [9]:
played_more_than_10_times.count()

2043582

# Part 2

In [10]:
business = spark.read.json("./yelp-dataset/yelp_academic_dataset_business.json")
reviews = spark.read.json("./yelp-dataset/yelp_academic_dataset_review.json")
users = spark.read.json("./yelp-dataset/yelp_academic_dataset_user.json")
business.createOrReplaceTempView("business")
reviews.createOrReplaceTempView("reviews")
users.createOrReplaceTempView("users")

21/12/01 13:49:23 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [11]:
# Query 1
spark.sql("select state, count(state) as count from business group by state order by count(state) desc").show()

+-----+-----+
|state|count|
+-----+-----+
|   AZ|56686|
|   NV|36312|
|   ON|33412|
|   NC|14720|
|   OH|14697|
|   PA|11216|
|   QC| 9219|
|   AB| 8012|
|   WI| 5154|
|   IL| 1932|
|   SC| 1162|
|   NY|   22|
|   CA|   19|
|   TX|    6|
|   FL|    4|
|  XGM|    4|
|   AL|    3|
|   WA|    3|
|   CT|    3|
|   VA|    2|
+-----+-----+
only showing top 20 rows



In [12]:
# Query 2
spark.sql("""
select count(distinct(*)) from (
    select explode(split(categories, \",\s*\")) as category from business
)
""").show()

+------------------------+
|count(DISTINCT category)|
+------------------------+
|                    2468|
+------------------------+



In [13]:
# Query 3
spark.sql("""
select category, count(category) from 
    (
        select explode(split(categories, \",\s*\")) as category 
        from business where city=\"Phoenix\"
    )
group by category order by count(category) desc limit 10
""").show()

+-----------------+---------------+
|         category|count(category)|
+-----------------+---------------+
|      Restaurants|           2815|
|         Shopping|           2416|
|    Home Services|           2302|
|             Food|           1672|
| Health & Medical|           1577|
|   Local Services|           1444|
|      Restaurants|           1184|
|       Automotive|           1164|
|    Beauty & Spas|           1115|
|    Home Services|            843|
+-----------------+---------------+



In [14]:
# Query 4 
spark.sql("""
select 
    count(*) as friend_count 
from 
    users 
where 
    size(split(friends, \",\s*\")) > 1000
""").show()

+------------+
|friend_count|
+------------+
|        4166|
+------------+



In [15]:
# Query 6
# spark.sql(
# """with p1 as 
#     (SELECT business_id, user_id, MAX(to_date(date))
#         FROM reviews GROUP BY user_id, business_id),
#     p2 as (SELECT business_id FROM business WHERE categories LIKE ('%Chinese%') 
#         AND categories LIKE ('%Restaurants%')
#     ),
#     p3 as (SELECT p1.user_id 
#         from p1 inner 
#         join p2 
#         on p2.business_id = p1.business_id)
#     SELECT COUNT(*) FROM users inner join p3 on p3.user_id = users.user_id
# """).show()

In [ ]:
#Query 6

spark.sql("""
with dates as (select max(to_date(date)) as date, user_id
from reviews
group by user_id)

select COUNT(f.*)
from reviews as r1
join dates as d
on d.user_id = r1.user_id

join business as b
on b.business_id = r1.business_id

join users as f
on array_contains(split(f.friends, \",\s*\"), r1.user_id)

where 
    date_trunc("DAY", r1.date) = date_trunc("DAY", d.date)
    and
    array_contains(split(b.categories, \",\s*\"), 'Chinese')

""").show()